# Creating the tree
Increasing indent means deeper rows

In [ ]:
class Pos:
    def __init__(self, val=0, name=""):
        self.value = val
        self.name = name
        self.children = []

    def add_child(self, node):
        self.children.append(node)

    def getPos(self):
        return (self.name, self.value)

    def correct_reorder_children(self, tree_depth=0):
        """
        To make minimax-alphabeta pruning works,
        sort Max Player's children nodes in descending order,
        sort Min Player's children nodes in ascending order

        recursive in coding is confusing. This function makes it double.
        In this example, self.children.sort does 2 things
            1st: returns value for A1=max(A11,A12,A13), A2=max(A21,A22,A23), A3=max(A31,A32,A33) by recursive call getValue()
            2nd: sorts A1,A2,A3 in descending order since root node is Max Player


        ` For sorted nodes [A1,A2,A3], visit each node and do the two processes at L35-L43 again for A1,A2,A3.
        That is what happening in at L45-L46
        """

        # Depth 0,2,4 has Max Player; Depth 1,3,5 has Min Player.
        # Starting from root node,
        # sort row 1,3,5... according for Max Player
        # sort row 0,2,4 according for Min Player. Bottom row in this exercise is row 2
        is_max_player = (tree_depth % 2 == 0)

        def getValue(pos):
            if pos.value is not None:
                return pos.value
            elif pos.children:
                list_children = [getValue(child) for child in pos.children]
                return max(list_children) if is_max_player else min(list_children)
            else:
                return -float('inf') if is_max_player else float('inf')
        self.children.sort(key=getValue, reverse=is_max_player)

        for child in self.children:
            child.correct_reorder_children(tree_depth=tree_depth + 1)

        return self

    def naive_reorder_child(self, reverse=False):
        def get_sort_value(pos):
            if pos.value is not None:
                return pos.value
            elif pos.children:
                sorted_children = [get_sort_value(child) for child in pos.children]
                return max(sorted_children) if reverse else min(sorted_children)
            else:
                return -float('inf') if reverse else float('inf')

        self.children.sort(key=get_sort_value, reverse=reverse)

        for child in self.children:
            child.naive_reorder_child(reverse=reverse)

currentPos = Pos(val=None, name="currentPos")
A1 = Pos(val=None, name="A1")
A2 = Pos(val=None, name="A2")
A3 = Pos(val=None, name="A3")
A11 = Pos(val=11, name="A11")
A12 = Pos(val=3, name="A12")
A13 = Pos(val=8, name="A13")
A21 = Pos(val=12, name="A21")
A22 = Pos(val=6, name="A22")
A23 = Pos(val=7, name="A23")
A31 = Pos(val=4, name="A31")
A32 = Pos(val=5, name="A32")
A33 = Pos(val=12, name="A33")

currentPos.add_child(A1)
currentPos.add_child(A2)
currentPos.add_child(A3)
A1.add_child(A11)
A1.add_child(A12)
A1.add_child(A13)
A2.add_child(A21)
A2.add_child(A22)
A2.add_child(A23)
A3.add_child(A31)
A3.add_child(A32)
A3.add_child(A33)

def print_tree(pos, level=0):
    indent = "  " * level
    print(f"{indent}{pos.getPos()}")

    for child in pos.children:
        print_tree(child, level + 1)

print_tree(currentPos)

('currentPos', None)
  ('A1', None)
    ('A11', 11)
    ('A12', 3)
    ('A13', 8)
  ('A2', None)
    ('A21', 12)
    ('A22', 6)
    ('A23', 7)
  ('A3', None)
    ('A31', 4)
    ('A32', 5)
    ('A33', 12)


# Minimax function
- Increasing indent means minimax search is reaching til the end.
- Trace code visualises which node is being visited by minimax first. So from the example trace code below, you read as "Minimax visits all nodes in the following orders: A11 -> A12 -> A13 -> A1 -> A21 -> A22 -> A23 -> A2 -> A31 -> A32 -> A33 -> A3 -> root node".
- Minimax depth search starts from bottom left nodes all the way to top (like inverted DFS) whereas we look at the tree from root node to bottom nodes. So to not confusing ourselves looking at the trace code, I define minimax depth as the exact row-ordering that minimax is visiting nodes: from bottom -> top, and tree depth is the ordering that we are looking at: from top -> bottom.

In [ ]:
def gameOver(pos):
    if pos.value == 100:
        return True
    else:
        return False


nodeProcessed = 0
def minimax(pos, minimax_depth, maxPlayer, tree_depth=0):
    global nodeProcessed

    # tracing minimax code
    indent = "  " * minimax_depth
    print(f"{indent} Visiting position: {pos.getPos()} | Minimax Depth: {minimax_depth} | Tree Depth: {tree_depth} | MaxPlayer: {maxPlayer}")

    if minimax_depth == 0 or gameOver(pos):
        return pos.value

    if maxPlayer:
        maxEval = -float("inf")
        for child in pos.children:
            nodeProcessed += 1
            eval = minimax(pos=child, minimax_depth=minimax_depth - 1, maxPlayer=False, tree_depth=tree_depth + 1)
            maxEval = max(maxEval, eval)
        print(f"{indent} Position: {pos.getPos()} update value to: {maxEval} \n")
        return maxEval

    else:
        minEval = float("inf")
        for child in pos.children:
            nodeProcessed += 1
            eval = minimax(pos=child, minimax_depth=minimax_depth - 1, maxPlayer=True, tree_depth=tree_depth + 1)
            minEval = min(minEval, eval)
        print(f"{indent} Position: {pos.getPos()} update value to: {minEval} \n")
        return minEval

rootPos = minimax(pos=currentPos, minimax_depth=2, maxPlayer=True)
print("\nMax Player's value at Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/12 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A11', 11) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A12', 3) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A13', 8) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A1', None) update value to: 3 

   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A21', 12) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A22', 6) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A23', 7) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 6 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 4) | Mi

# Minimax AlphaBeta pruning function with code tracing like above
- Minimax AlphaBeta doesn't start pruning until alpha and beta are being updated from their init -inf, +inf values respectively. Recall that Minimax acts like an inverted DFS, thus this updating processing doesn't happen until all bottom left nodes are visited.
- This is why say you have [A21=12, A22=6, A23=7], A21 cannot be pruned instantly as we intuitively do in our head.


In [ ]:
def minimax_alphabeta(pos, minimax_depth, maxPlayer, alpha, beta, tree_depth=0):
    global nodeProcessed

    # tracing minimax with alpha-beta pruning code
    indent = "  " * minimax_depth
    print(f"{indent} Visiting position: {pos.getPos()} | Minimax Depth: {minimax_depth} | Tree Depth: {tree_depth} | MaxPlayer: {maxPlayer}")

    if minimax_depth == 0 or gameOver(pos):
        return pos.value

    if maxPlayer:
        maxEval = -float("inf")
        for child in pos.children:
            nodeProcessed += 1
            eval = minimax_alphabeta(pos=child, minimax_depth=minimax_depth - 1, maxPlayer=False, alpha=alpha, beta=beta, tree_depth=tree_depth + 1)
            maxEval = max(maxEval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        print(f"{indent} Position: {pos.getPos()} update value to: {maxEval} \n")
        return maxEval

    else:
        minEval = float("inf")
        for child in pos.children:
            nodeProcessed += 1
            eval = minimax_alphabeta(pos=child, minimax_depth=minimax_depth - 1, maxPlayer=True, alpha=alpha, beta=beta, tree_depth=tree_depth + 1)
            minEval = min(minEval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        print(f"{indent} Position: {pos.getPos()} update value to: {minEval} \n")
        return minEval

# Minimax AlphaBeta pruning with no sorting bottom nodes

In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\nMax Player's value at Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/12 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A11', 11) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A12', 3) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A13', 8) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A1', None) update value to: 3 

   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A21', 12) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A22', 6) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A23', 7) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 6 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 4) | Mi

# Minimax AlphaBeta pruning with sorting Min Player's children nodes in ascending order and Max Player's children nodes in descending order

In [ ]:
currentPos.correct_reorder_children()
print_tree(currentPos)

('currentPos', None)
  ('A2', None)
    ('A22', 6)
    ('A23', 7)
    ('A21', 12)
  ('A3', None)
    ('A31', 4)
    ('A32', 5)
    ('A33', 12)
  ('A1', None)
    ('A12', 3)
    ('A13', 8)
    ('A11', 11)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\n Max evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} node processed/12 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A22', 6) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A23', 7) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A21', 12) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 6 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 4) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A3', None) update value to: 4 

   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A12', 3) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A1', None) update value to: 3 

     Position: ('currentPos', None) update value to: 6 


 Max evaluation

# Only sort bottom nodes in ascending/descending order

In [ ]:
currentPos.naive_reorder_child(reverse=False)
print_tree(currentPos)

('currentPos', None)
  ('A1', None)
    ('A12', 3)
    ('A13', 8)
    ('A11', 11)
  ('A3', None)
    ('A31', 4)
    ('A32', 5)
    ('A33', 12)
  ('A2', None)
    ('A22', 6)
    ('A23', 7)
    ('A21', 12)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\n Max evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/12 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A12', 3) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A13', 8) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A11', 11) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A1', None) update value to: 3 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 4) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A32', 5) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A33', 12) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A3', None) update value to: 4 

   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A22', 6) | Mi

# Manual reordering

In [ ]:
currentPos.correct_reorder_children()
currentPos.children = [A3, A2, A1]
print_tree(currentPos)

('currentPos', None)
  ('A3', None)
    ('A31', 4)
    ('A32', 5)
    ('A33', 12)
  ('A2', None)
    ('A22', 6)
    ('A23', 7)
    ('A21', 12)
  ('A1', None)
    ('A12', 3)
    ('A13', 8)
    ('A11', 11)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\n Max evaluation for Current Position (root node): ", rootPos)
print("# next positions processed out of total 12 next positions: ", nodeProcessed)

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 4) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A32', 5) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A33', 12) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A3', None) update value to: 4 

   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A22', 6) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A23', 7) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A21', 12) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 6 

   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A12', 3) | Mi

# Other examples


In [ ]:
currentPos = Pos(val=None, name="currentPos")
A1 = Pos(val=None, name="A1")
A2 = Pos(val=None, name="A2")
A3 = Pos(val=None, name="A3")
A11 = Pos(val=11, name="A11")
A12 = Pos(val=13, name="A12")
A13 = Pos(val=8, name="A13")
A21 = Pos(val=12, name="A21")
A22 = Pos(val=16, name="A22")
A23 = Pos(val=17, name="A23")
A31 = Pos(val=14, name="A31")
A32 = Pos(val=16, name="A32")

currentPos.add_child(A1)
currentPos.add_child(A2)
currentPos.add_child(A3)
A1.add_child(A11)
A1.add_child(A12)
A1.add_child(A13)
A2.add_child(A21)
A2.add_child(A22)
A2.add_child(A23)
A3.add_child(A31)
A3.add_child(A32)

def print_tree(pos, level=0):
    indent = "  " * level
    print(f"{indent}{pos.getPos()}")

    for child in pos.children:
        print_tree(child, level + 1)

print_tree(currentPos)

('currentPos', None)
  ('A1', None)
    ('A11', 11)
    ('A12', 13)
    ('A13', 8)
  ('A2', None)
    ('A21', 12)
    ('A22', 16)
    ('A23', 17)
  ('A3', None)
    ('A31', 14)
    ('A32', 16)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\nMax evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/11 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A11', 11) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A12', 13) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A13', 8) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A1', None) update value to: 8 

   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A21', 12) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A22', 16) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A23', 17) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 12 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 14)

In [ ]:
currentPos.correct_reorder_children()
print_tree(currentPos)

('currentPos', None)
  ('A2', None)
    ('A21', 12)
    ('A22', 16)
    ('A23', 17)
  ('A3', None)
    ('A31', 14)
    ('A32', 16)
  ('A1', None)
    ('A13', 8)
    ('A11', 11)
    ('A12', 13)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\nMax evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/11 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A11', 11) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A12', 7) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A13', 9) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A1', None) update value to: 7 

   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A21', 2) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 2 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 6) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A3', None) update value to: 6 

     Position: ('currentPos', None) update value to: 7 


Max evaluation 

In [ ]:
currentPos = Pos(val=None, name="currentPos")
A1 = Pos(val=None, name="A1")
A2 = Pos(val=None, name="A2")
A3 = Pos(val=None, name="A3")
A11 = Pos(val=11, name="A11")
A12 = Pos(val=7, name="A12")
A13 = Pos(val=9, name="A13")
A21 = Pos(val=2, name="A21")
A22 = Pos(val=16, name="A22")
A23 = Pos(val=17, name="A23")
A31 = Pos(val=6, name="A31")
A32 = Pos(val=12, name="A32")
A33 = Pos(val=2, name="A33")
A34 = Pos(val=9, name="A34")

currentPos.add_child(A1)
currentPos.add_child(A2)
currentPos.add_child(A3)
A1.add_child(A11)
A1.add_child(A12)
A1.add_child(A13)
A2.add_child(A21)
A2.add_child(A22)
A2.add_child(A23)
A3.add_child(A31)
A3.add_child(A32)
A3.add_child(A33)
A3.add_child(A34)

def print_tree(pos, level=0):
    indent = "  " * level
    print(f"{indent}{pos.getPos()}")

    for child in pos.children:
        print_tree(child, level + 1)

print_tree(currentPos)

('currentPos', None)
  ('A1', None)
    ('A11', 11)
    ('A12', 7)
    ('A13', 9)
  ('A2', None)
    ('A21', 2)
    ('A22', 16)
    ('A23', 17)
  ('A3', None)
    ('A31', 6)
    ('A32', 12)
    ('A33', 2)
    ('A34', 9)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\nMax evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/13 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A11', 11) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A12', 7) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A13', 9) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A1', None) update value to: 7 

   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A21', 2) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 2 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A31', 6) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A3', None) update value to: 6 

     Position: ('currentPos', None) update value to: 7 


Max evaluation 

In [ ]:
currentPos.correct_reorder_children()
print_tree(currentPos)

('currentPos', None)
  ('A2', None)
    ('A21', 2)
    ('A22', 16)
    ('A23', 17)
  ('A3', None)
    ('A33', 2)
    ('A31', 6)
    ('A34', 9)
    ('A32', 12)
  ('A1', None)
    ('A12', 7)
    ('A13', 9)
    ('A11', 11)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=2, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\nMax evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/13 total nodes")

     Visiting position: ('currentPos', None) | Minimax Depth: 2 | Tree Depth: 0 | MaxPlayer: True
   Visiting position: ('A2', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A21', 2) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A22', 16) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A23', 17) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A2', None) update value to: 2 

   Visiting position: ('A3', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A33', 2) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
   Position: ('A3', None) update value to: 2 

   Visiting position: ('A1', None) | Minimax Depth: 1 | Tree Depth: 1 | MaxPlayer: False
 Visiting position: ('A12', 7) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A13', 9) | Minimax Depth: 0 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A11', 11) | M

In [ ]:
currentPos = Pos(val=None, name="currentPos")
A1 = Pos(val=None, name="A1")
A2 = Pos(val=None, name="A2")
A11 = Pos(val=None, name="A11")
A12 = Pos(val=None, name="A12")
A21 = Pos(val=None, name="A21")
A22 = Pos(val=None, name="A22")
A111 = Pos(val=-2, name="A111")
A112 = Pos(val=-3, name="A112")
A121 = Pos(val=-4, name="A121")
A122 = Pos(val=-5, name="A122")
A211 = Pos(val=3, name="A211")
A212 = Pos(val=2, name="A212")
A221 = Pos(val=-5, name="A221")
A222 = Pos(val=-3, name="A222")

currentPos.add_child(A1)
currentPos.add_child(A2)
A1.add_child(A11)
A1.add_child(A12)
A2.add_child(A21)
A2.add_child(A22)
A11.add_child(A111)
A11.add_child(A112)
A12.add_child(A121)
A12.add_child(A122)
A21.add_child(A211)
A21.add_child(A212)
A22.add_child(A221)
A22.add_child(A222)

def print_tree(pos, level=0):
    indent = "  " * level
    print(f"{indent}{pos.getPos()}")

    for child in pos.children:
        print_tree(child, level + 1)

print_tree(currentPos)

('currentPos', None)
  ('A1', None)
    ('A11', None)
      ('A111', -2)
      ('A112', -3)
    ('A12', None)
      ('A121', -4)
      ('A122', -5)
  ('A2', None)
    ('A21', None)
      ('A211', 3)
      ('A212', 2)
    ('A22', None)
      ('A221', -5)
      ('A222', -3)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=3, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\nMax evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/14 total nodes")

       Visiting position: ('currentPos', None) | Minimax Depth: 3 | Tree Depth: 0 | MaxPlayer: True
     Visiting position: ('A1', None) | Minimax Depth: 2 | Tree Depth: 1 | MaxPlayer: False
   Visiting position: ('A11', None) | Minimax Depth: 1 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A111', -2) | Minimax Depth: 0 | Tree Depth: 3 | MaxPlayer: False
 Visiting position: ('A112', -3) | Minimax Depth: 0 | Tree Depth: 3 | MaxPlayer: False
   Position: ('A11', None) update value to: -2 

   Visiting position: ('A12', None) | Minimax Depth: 1 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A121', -4) | Minimax Depth: 0 | Tree Depth: 3 | MaxPlayer: False
 Visiting position: ('A122', -5) | Minimax Depth: 0 | Tree Depth: 3 | MaxPlayer: False
   Position: ('A12', None) update value to: -4 

     Position: ('A1', None) update value to: -4 

     Visiting position: ('A2', None) | Minimax Depth: 2 | Tree Depth: 1 | MaxPlayer: False
   Visiting position: ('A21', None) | Minima

In [ ]:
currentPos.correct_reorder_children()
print_tree(currentPos)

('currentPos', None)
  ('A2', None)
    ('A22', None)
      ('A222', -3)
      ('A221', -5)
    ('A21', None)
      ('A211', 3)
      ('A212', 2)
  ('A1', None)
    ('A12', None)
      ('A121', -4)
      ('A122', -5)
    ('A11', None)
      ('A111', -2)
      ('A112', -3)


In [ ]:
nodeProcessed = 0
rootPos = minimax_alphabeta(pos=currentPos, minimax_depth=3, maxPlayer=True, alpha=-float("inf"), beta=float("inf"))
print("\nMax evaluation for Current Position (root node): ", rootPos)
print(f"{nodeProcessed} nodes processed/14 total nodes")

       Visiting position: ('currentPos', None) | Minimax Depth: 3 | Tree Depth: 0 | MaxPlayer: True
     Visiting position: ('A2', None) | Minimax Depth: 2 | Tree Depth: 1 | MaxPlayer: False
   Visiting position: ('A22', None) | Minimax Depth: 1 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A222', -3) | Minimax Depth: 0 | Tree Depth: 3 | MaxPlayer: False
 Visiting position: ('A221', -5) | Minimax Depth: 0 | Tree Depth: 3 | MaxPlayer: False
   Position: ('A22', None) update value to: -3 

   Visiting position: ('A21', None) | Minimax Depth: 1 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A211', 3) | Minimax Depth: 0 | Tree Depth: 3 | MaxPlayer: False
   Position: ('A21', None) update value to: 3 

     Position: ('A2', None) update value to: -3 

     Visiting position: ('A1', None) | Minimax Depth: 2 | Tree Depth: 1 | MaxPlayer: False
   Visiting position: ('A12', None) | Minimax Depth: 1 | Tree Depth: 2 | MaxPlayer: True
 Visiting position: ('A121', -4) | Minimax D